1. 보험 약관 챗봇 에이전트 (RAG-약관)
2. 학습 조교 에이전트 (RAG-수업자료)
3. 자유주제 에이전트

In [ ]:
# 비즈니스 이메일 작성 에이전트
# Business Email Writing Agent

import openai
import re
from datetime import datetime
from typing import Dict, List, Optional

class BusinessEmailAgent:
    def __init__(self, api_key: str = None):
        """
        비즈니스 이메일 작성 에이전트 초기화
        
        Args:
            api_key: OpenAI API 키 (또는 다른 LLM API 키)
        """
        if api_key:
            openai.api_key = api_key
        
        # 이메일 템플릿 및 톤 설정
        self.email_templates = {
            "meeting_request": "회의 요청",
            "follow_up": "후속 연락", 
            "proposal": "제안서",
            "inquiry": "문의",
            "complaint": "불만 처리",
            "thank_you": "감사 인사",
            "introduction": "소개/인사",
            "negotiation": "협상"
        }
        
        self.tone_options = {
            "formal": "격식있는",
            "friendly": "친근한", 
            "professional": "전문적인",
            "urgent": "긴급한",
            "apologetic": "사과하는"
        }
    
    def analyze_request(self, user_input: str) -> Dict:
        """
        사용자 요청 분석
        
        Args:
            user_input: 사용자가 입력한 이메일 작성 요청
            
        Returns:
            분석된 요청 정보
        """
        # 키워드 기반 분석 (실제로는 NLP 모델 사용 권장)
        request_info = {
            "type": "general",
            "tone": "professional", 
            "urgency": "normal",
            "recipient": "",
            "main_points": [],
            "deadline": None
        }
        
        # 이메일 타입 감지
        for template_key, template_name in self.email_templates.items():
            if any(keyword in user_input.lower() for keyword in [
                template_key.replace("_", " "), template_name
            ]):
                request_info["type"] = template_key
                break
        
        # 톤 감지
        if any(word in user_input.lower() for word in ["급한", "긴급", "빨리"]):
            request_info["tone"] = "urgent"
        elif any(word in user_input.lower() for word in ["죄송", "미안", "사과"]):
            request_info["tone"] = "apologetic"
        elif any(word in user_input.lower() for word in ["친근", "편안"]):
            request_info["tone"] = "friendly"
        
        return request_info
    
    def generate_email_structure(self, request_info: Dict) -> Dict:
        """
        이메일 구조 생성
        
        Args:
            request_info: 분석된 요청 정보
            
        Returns:
            이메일 구조
        """
        structure = {
            "subject": "",
            "greeting": "",
            "opening": "",
            "body": "",
            "closing": "",
            "signature": ""
        }
        
        # 템플릿별 구조 설정
        if request_info["type"] == "meeting_request":
            structure["subject"] = "회의 일정 조율 요청"
            structure["opening"] = "회의 일정에 대해 문의드리고자 연락드립니다."
            
        elif request_info["type"] == "follow_up":
            structure["subject"] = "후속 조치 관련 문의"
            structure["opening"] = "이전에 논의된 사항에 대한 진행 상황을 확인하고자 합니다."
            
        # 더 많은 템플릿 추가 가능
        
        return structure
    
    def compose_email(self, request_info: Dict, structure: Dict, 
                     custom_content: Dict = None) -> str:
        """
        최종 이메일 작성
        
        Args:
            request_info: 요청 정보
            structure: 이메일 구조
            custom_content: 사용자 커스텀 내용
            
        Returns:
            완성된 이메일
        """
        email_parts = []
        
        # 제목
        email_parts.append(f"제목: {structure['subject']}")
        email_parts.append("")
        
        # 인사말
        greeting = self.get_appropriate_greeting(request_info["tone"])
        email_parts.append(greeting)
        email_parts.append("")
        
        # 본문 시작
        email_parts.append(structure["opening"])
        email_parts.append("")
        
        # 메인 내용
        if custom_content and "main_content" in custom_content:
            email_parts.append(custom_content["main_content"])
        else:
            email_parts.append(self.generate_default_content(request_info))
        
        email_parts.append("")
        
        # 마무리
        closing = self.get_appropriate_closing(request_info["tone"])
        email_parts.append(closing)
        email_parts.append("")
        
        # 서명
        signature = custom_content.get("signature", "감사합니다.") if custom_content else "감사합니다."
        email_parts.append(signature)
        
        return "\n".join(email_parts)
    
    def get_appropriate_greeting(self, tone: str) -> str:
        """톤에 맞는 인사말 반환"""
        greetings = {
            "formal": "안녕하십니까.",
            "friendly": "안녕하세요!",
            "professional": "안녕하세요.",
            "urgent": "안녕하세요.",
            "apologetic": "안녕하세요."
        }
        return greetings.get(tone, "안녕하세요.")
    
    def get_appropriate_closing(self, tone: str) -> str:
        """톤에 맞는 마무리말 반환"""
        closings = {
            "formal": "검토 후 회신 부탁드립니다.",
            "friendly": "좋은 답변 기다릴게요!",
            "professional": "확인 후 연락 부탁드립니다.",
            "urgent": "빠른 시일 내 답변 부탁드립니다.",
            "apologetic": "불편을 드려 죄송합니다."
        }
        return closings.get(tone, "확인 후 연락 부탁드립니다.")
    
    def generate_default_content(self, request_info: Dict) -> str:
        """기본 내용 생성"""
        if request_info["type"] == "meeting_request":
            return "다음 주 중 가능한 시간에 회의를 진행하고자 합니다.\n가능한 일정을 알려주시면 조율하겠습니다."
        elif request_info["type"] == "follow_up":
            return "이전에 논의된 사항의 현재 진행 상황을 공유해 주시면 감사하겠습니다."
        else:
            return "상세한 내용은 첨부 파일을 참고해 주세요."
    
    def create_email(self, user_request: str, custom_content: Dict = None) -> str:
        """
        메인 이메일 생성 함수
        
        Args:
            user_request: 사용자 요청
            custom_content: 커스텀 내용
            
        Returns:
            완성된 이메일
        """
        # 1단계: 요청 분석
        request_info = self.analyze_request(user_request)
        
        # 2단계: 이메일 구조 생성
        structure = self.generate_email_structure(request_info)
        
        # 3단계: 이메일 작성
        email = self.compose_email(request_info, structure, custom_content)
        
        return email

# 사용 예시
def demo_usage():
    """에이전트 사용 데모"""
    agent = BusinessEmailAgent()
    
    # 예시 1: 회의 요청 이메일
    request1 = "다음 주에 프로젝트 회의 잡고 싶어요"
    email1 = agent.create_email(request1)
    print("=== 회의 요청 이메일 ===")
    print(email1)
    print("\n" + "="*50 + "\n")
    
    # 예시 2: 커스텀 내용이 있는 이메일
    request2 = "클라이언트에게 제안서 관련 후속 연락"
    custom_content = {
        "main_content": "지난주 제출한 제안서에 대한 검토 결과를 문의드립니다.\n추가로 필요한 자료가 있다면 언제든 말씀해 주세요.",
        "signature": "최고의 서비스로 보답하겠습니다.\n\n김대리 드림\n마케팅팀\n010-1234-5678"
    }
    email2 = agent.create_email(request2, custom_content)
    print("=== 커스텀 후속 연락 이메일 ===")
    print(email2)

if __name__ == "__main__":
    demo_usage()